<a href="https://colab.research.google.com/github/JF11579/Plotly_RE/blob/main/SameHouse/Lot_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

### Way too many missing land size data afor this to bemeaningful

# Libraries

In [1]:
import pandas as pd
from google.colab import drive  # <<<<<<<<<------
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Options

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data

In [28]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/Real_Estate_Data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DATA/Real_Estate_Data


In [30]:
houses = pd.read_csv('Combined_data_Februrary_18.csv')

<ipython-input-30-dee981abd164>:1: DtypeWarning:

Columns (19,21,23,24,25,26,28,30,31,34,35,37,38,39,42,43,45,47,48,49,50,51,53,56,57,58,59,61,62,64,74,75,76,78,87,88,91,92,104,106,109,111,113,115,117,122,123,124,125,127,135,142,143,144,151,152,154,156,158,160,161,164,167,174,180,181,184,185,187,188,189,190,191,192,193,194,199,200,202,203,208,214,216,217,220,221,222,223,227,228,231,232,233,234,235,237) have mixed types. Specify dtype option on import or set low_memory=False.



# PRe Process

In [31]:
#  'Lot_Square_Footage', 'Lot_Size',

data = houses

data = data.dropna(subset=["Sold_Price"]).query("`Sold_Price` != 0")
data = data[['Address', 'Town','Zip','State','Style',
                    'ML#','Sold_Price','Baths_Full', 'Baths_Half',
                    'Bath','Bedrooms',
                      'Appearance','REO' , 'School_District_Name',
                 'Approx_Int_Square_Footage','Street_Name','Contract_Date', 'Lot_Square_Footage', 'Lot_Size']]


data= data.loc[data.Town.isin(['Roosevelt', 'Hempstead', 'Freeport', 'Farmingdale', 'Mineola','Westbury', 'Roslyns', 'Port Washington', 'Locust Valley'])]


# Convert 'Contract_Date' to datetime
data['Contract_Date'] = pd.to_datetime(data['Contract_Date'])

# Extract the year
data['Year'] = data['Contract_Date'].dt.year


# Drop rows where 'Year' is NaN
data = data.dropna(subset=['Year'])

# Now convert 'Year' to integer
#data['Year'] = data['Year'].astype(int)
####
median_sold_price_all = data.groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_price_all.rename(columns={'Sold_Price': 'Median_Sold_Price_All'}, inplace=True)
#
median_sold_non_ranch = data[data['Style'] != 'Ranch'].groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_non_ranch.rename(columns={'Sold_Price': 'Median_Sold_Non_Ranch'}, inplace=True)
#
median_sold_ranch = data[data['Style'] == 'Ranch'].groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_ranch.rename(columns={'Sold_Price': 'Median_Sold_Ranch'}, inplace=True)

#######
# Merge Median_Sold_Price_All
data = data.merge(median_sold_price_all, on=['Town', 'Year'], how='left')

# Merge Median_Sold_Non_Ranch
data = data.merge(median_sold_non_ranch, on=['Town', 'Year'], how='left')

# Merge Median_Sold_Ranch
data = data.merge(median_sold_ranch, on=['Town', 'Year'], how='left')

houses = data

houses['Month_Year'] = houses['Contract_Date'].dt.strftime('%B-%Y')

In [32]:
houses.head()

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Lot_Square_Footage,Lot_Size,Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch,Month_Year
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.00,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,7500.0,50 X 150,2006.0,609500.0,610000.0,520000.0,September-2006
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.00,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,5564.0,57 X 107,2015.0,485000.0,515000.0,440000.0,November-2015
2,19 White Spots Lane,Locust Valley,11560,NY,Exp Cape,2427641,650000.0,2.0,1.0,2.50,3.0,Mint,N,Locust Valley,2353.0,White Spots,2011-10-08,10492.0,0.24,2011.0,666000.0,770000.0,361000.0,October-2011
3,5 Quarry Road,Locust Valley,11560,NY,Exp Ranch,2396870,800000.0,3.0,0.0,3.00,4.0,Excellent,N,Locust Valley,2151.0,Quarry,2012-05-03,27007.0,.62,2012.0,550000.0,512500.0,717800.0,May-2012
4,3 Nassau Road,Locust Valley,11560,NY,Colonial,2030725,467500.0,2.0,2.0,2.55,4.0,Diamond!!,N,Locust Valley,NaN,Nassau,2008-06-22,6098.0,62X100,2008.0,545000.0,582500.0,302500.0,June-2008


In [33]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26087 entries, 0 to 26086
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Address                    26087 non-null  object        
 1   Town                       26087 non-null  object        
 2   Zip                        26087 non-null  int64         
 3   State                      26087 non-null  object        
 4   Style                      26085 non-null  object        
 5   ML#                        26087 non-null  object        
 6   Sold_Price                 26087 non-null  float64       
 7   Baths_Full                 26087 non-null  float64       
 8   Baths_Half                 26086 non-null  float64       
 9   Bath                       26080 non-null  float64       
 10  Bedrooms                   26087 non-null  float64       
 11  Appearance                 20299 non-null  object        
 12  REO 

In [49]:
import plotly.graph_objects as go

# Assuming 'category_counts' is your DataFrame prepared as before

# Initialize an empty figure
fig = go.Figure()

# Keep track of towns added to the legend
added_towns = set()

# Iterate over each row in the DataFrame to add traces
for _, row in category_counts.iterrows():
    # Determine whether to show in legend
    show_in_legend = row['Town'] not in added_towns
    if show_in_legend:
        # Mark the town as added
        added_towns.add(row['Town'])

    # Add trace to the figure
    fig.add_trace(go.Bar(
        name=row['Town'],
        x=[row['Lot_Size_Category']],
        y=[row['Percentage']],
        marker_color=color_dict[row['Town']],
        text=f"{row['Count']}",
        textposition='inside',
        showlegend=show_in_legend  # Control legend entry
    ))

# Customize layout
fig.update_layout(
    barmode='stack',
    title="Percentage of Properties by Lot Size Category and Town",
    xaxis_title='Lot Size Category',
    yaxis_title='Percentage',
    legend_title='Town'
)

# Show the plot
fig.show()


In [47]:
import plotly.graph_objects as go

# Assuming 'category_counts' is your DataFrame prepared as before

# Initialize an empty figure
fig = go.Figure()

# Keep track of towns added to the legend
added_towns = set()

# Iterate over each row in the DataFrame to add traces
for _, row in category_counts.iterrows():
    # Determine whether to show in legend
    show_in_legend = row['Town'] not in added_towns
    if show_in_legend:
        # Mark the town as added
        added_towns.add(row['Town'])

    # Add trace to the figure
    fig.add_trace(go.Bar(
        name=row['Town'],
        x=[row['Lot_Size_Category']],
        y=[row['Percentage']],
        marker_color=color_dict[row['Town']],
        text=f"{row['Count']}",
        textposition='inside',
        showlegend=show_in_legend  # Control legend entry
    ))

# Customize layout
fig.update_layout(
    barmode='stack',
    title="Percentage of Properties by Lot Size Category and Town",
    xaxis_title='Lot Size Category',
    yaxis_title='Percentage',
    legend_title='Town'
)

# Show the plot
fig.show()


In [48]:
import plotly.graph_objects as go

# Assuming 'category_counts' is your DataFrame prepared as before

# Initialize an empty figure
fig = go.Figure()

# Keep track of towns added to the legend
added_towns = set()

# Iterate over each row in the DataFrame to add traces
for _, row in category_counts.iterrows():
    # Determine whether to show in legend
    show_in_legend = row['Town'] not in added_towns
    if show_in_legend:
        # Mark the town as added
        added_towns.add(row['Town'])

    # Add trace to the figure
    fig.add_trace(go.Bar(
        name=row['Town'],
        x=[row['Lot_Size_Category']],
        y=[row['Percentage']],
        marker_color=color_dict[row['Town']],
        text=f"{row['Count']}",
        textposition='inside',
        showlegend=show_in_legend  # Control legend entry
    ))

# Customize layout
fig.update_layout(
    barmode='stack',
    title="Percentage of Properties by Lot Size Category and Town",
    xaxis_title='Lot Size Category',
    yaxis_title='Percentage',
    legend_title='Town'
)

# Show the plot
fig.show()


In [46]:
# Create a stacke bar plot
import plotly.graph_objects as go
import pandas as pd

# Assuming 'generic' is your prepared DataFrame with 'Lot_Size_Category' and 'Town'

# Calculate counts for each 'Lot_Size_Category' within each 'Town'
category_counts = generic.groupby(['Lot_Size_Category', 'Town']).size().reset_index(name='Count')

# Calculate the total counts in each 'Lot_Size_Category'
totals = category_counts.groupby('Lot_Size_Category')['Count'].sum().reset_index(name='Total')

# Merge to get total counts back into the original DataFrame
category_counts = category_counts.merge(totals, on='Lot_Size_Category')

# Calculate the percentage of each town's contribution to the total
category_counts['Percentage'] = (category_counts['Count'] / category_counts['Total']) * 100

# Initialize an empty figure
fig = go.Figure()

# Assign colors to each town
color_dict = {
    'Roslyns': '#006400',  # DarkGreen
    'Port Washington': '#008000',  # Green
    'Locust Valley': '#90EE90',  # LightGreen
    'Mineola': '#00008B',  # DarkBlue
    'Westbury': '#0000FF',  # Blue
    'Farmingdale': '#ADD8E6',  # LightBlue
    'Freeport': '#8B0000',  # DarkRed
    'Hempstead': '#FF0000',  # Red
    'Roosevelt': '#F08080'  # LightCoral
}

# Add a bar for each town within each lot size category
for category in category_counts['Lot_Size_Category'].unique():
    category_data = category_counts[category_counts['Lot_Size_Category'] == category]
    for _, row in category_data.iterrows():
        fig.add_trace(go.Bar(
            name=row['Town'],
            x=[category],
            y=[row['Percentage']],
            marker_color=color_dict[row['Town']],
            text=f"{row['Count']}",
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',  # Use 'stack' to stack bars
    title="Percentage of Properties by Lot Size Category and Town",
    xaxis_title='Lot Size Category',
    yaxis_title='Percentage',
    legend_title='Town'
)

# Show the plot
fig.show()


In [41]:
import pandas as pd
generic = houses

# Assuming 'generic' is your DataFrame
# Define the bins edges
bins = [0, 4000, 8000, 10000, 20000, 43451, 86900, float('inf')]
# Define the labels for the bins
labels = ['0-4000', '4001-8000', '8001-10000', '10001-20000', '20001-43451', '43452-86900', '86901+']

# Use pd.cut to categorize the lot sizes into bins
generic['Lot_Size_Category'] = pd.cut(generic['Lot_Square_Footage'], bins=bins, labels=labels, right=False)

# Check the distribution
print(generic['Lot_Size_Category'].value_counts())


4001-8000      17051
10001-20000     3574
8001-10000      3366
0-4000          1218
20001-43451      509
43452-86900      221
86901+           139
Name: Lot_Size_Category, dtype: int64


In [42]:
import plotly.express as px

# Create a histogram
fig = px.histogram(generic, x='Lot_Size_Category', title='Distribution of Properties by Lot Size Categories')

# Update layout if necessary
fig.update_layout(xaxis_title='Lot Size Category', yaxis_title='Count', bargap=0.2)

# Show the plot
fig.show()


In [34]:
import plotly.express as px

data = houses

# Assuming the DataFrame is correctly named med_incomes
# Ensure the town names in your DataFrame match these keys exactly, e.g., 'Port_Washington' instead of 'Port Washington'
color_dict = {
    'Roslyns': '#006400',  # DarkGreen
    'Port_Washington': '#008000',  # Green
    'Locust_Valley': '#90EE90',  # LightGreen
    'Mineola': '#00008B',  # DarkBlue
    'Westbury': '#0000FF',  # Blue
    'Farmingdale': '#ADD8E6',  # LightBlue
    'Freeport': '#8B0000',  # DarkRed
    'Hempstead': '#FF0000',  # Red
    'Roosevelt': '#F08080'  # LightCoral
}

# Create the scatter plot
fig = px.scatter(houses, x='Lot_Square_Footage', y='Sold_Price',
                 color = "Town",
                 color_discrete_map=color_dict,
                 hover_name="Town",
                 hover_data=["Lot_Square_Footage", "Sold_Price"],
                 title='Sold Price vs Lot Square Footage')



fig.update_layout(
    title="Average Household Income by Town and Year",
    xaxis_title="Year",
    yaxis_title="Average Household Income",
    legend_title="Town"
)

fig.show()


In [35]:
import pandas as pd
import plotly.express as px

# Assuming df is your DataFrame with 'Lot_Square_Footage' and 'Sold_Price' columns
# df = pd.read_csv('your_data.csv')  # Example to load data from a CSV file

# Create the scatter plot
fig = px.scatter(houses, x='Lot_Square_Footage', y='Sold_Price', title='Sold Price vs Lot Square Footage')

# Update layout with specific y-axis range
fig.update_layout(
    xaxis_title='Lot Square Footage',
    yaxis_title='Sold Price',
    plot_bgcolor='white',
    xaxis=dict(showline=True, showgrid=False, linecolor='black'),
    yaxis=dict(showline=True, showgrid=True, gridcolor='lightgrey', linecolor='black', range=[0, 3000000])  # Y-axis range set to 0 to $5 million
)

# Show the figure
fig.show()


In [36]:
# prompt: CAN I SEE ALL of th unique of Town

houses['Town'].unique()


array(['Locust Valley', 'Mineola', 'Farmingdale', 'Hempstead', 'Freeport',
       'Roslyns', 'Roosevelt', 'Westbury', 'Port Washington'],
      dtype=object)

In [37]:
# prompt: make a pltly histogram of  Lot_Square_Footage

fig = px.histogram(houses, x="Lot_Square_Footage", nbins=20, title="Distribution of Lot Square Footage")

# Update layout with specific y-axis range
fig.update_layout(
    xaxis_title='Lot Square Footage',
    yaxis_title='Frequency',
    plot_bgcolor='white',
    xaxis=dict(showline=True, showgrid=False, linecolor='black'),
    yaxis=dict(showline=True, showgrid=True, gridcolor='lightgrey', linecolor='black')
)

# Show the figure
fig.show()


In [38]:
# Filter the DataFrame to exclude rows where Town is 'Freeport'
# and then check if there are any non-null entries in 'Lot_Square_Footage'
non_freeport_lot_sqft_not_null = data[data['Town'] != 'Freeport']['Lot_Square_Footage'].notnull().any()

print(non_freeport_lot_sqft_not_null)


True
